# SIMULATION

In [1]:
import numpy as np
import pandas as pd
from numba import jit,njit, prange, objmode
import threading
import concurrent.futures
import warnings
import math
from scipy.optimize import minimize_scalar

warnings.filterwarnings('ignore') 


In [2]:
# Real simulation
@njit(fastmath=True,parallel=True,cache=True)
def add_error_jit(alpha_, variants_matrix): 
    return (variants_matrix - 2*alpha_*variants_matrix) + alpha_
    
class Population:
    def __init__(self, size=10000, nb_variants=4, nb_mutation=40, min_hfm=5, max_hfm=10):
        self.size = size  # represent the total number of read
        self.nb_variants = nb_variants
        self.nb_mutation = nb_mutation
        self.ratio = []
        # min_hfm,max_hfm = the number of mutation we assume that each variant has with high freq
        self.min_hfm = min_hfm
        self.max_hfm = max_hfm
        

        
    def generate_ratio(self):  # Randomely generate ratio of each variant
        self.ratio = np.random.dirichlet(np.ones(self.nb_variants) / 1.1, size=1)[0]
        return self.ratio
    
    def generate_degradation_profile(self):
        #degradation_deg: value from 0 to 10; 0 mean that there is no degradation
        self.degradation_profile = np.full(self.nb_mutation, 0.9) #initialization
        #randomly select wich mutations are degraded in fonction of degradation_deg:
        self.degradation_profile[np.random.choice(range(self.nb_mutation), int(self.degradation_deg/10*self.nb_mutation), replace=False)] = 0.9 + self.degradation_deg/100
        return self.degradation_profile
        
    def creat_BAM(self, alpha=0., degradation_deg=0):
        ############# Creat variants_matrix M ###########
        # variants_matrix is a nb_mutation x nb_variants matrix where each row represent a specific mutations
        # and each col represent a variant and the value of variants_matrix[i, j] is the likelhood that
        # the variation j will have the mutation i, there for it's between 0 and 1.
        self.variants_matrix = np.random.rand(self.nb_mutation, self.nb_variants) / 10  # we initialise it randomly with value between 0 and 0.1 to represente the noise
        ## attribute randomly selected mutation to a variant
        for j in range(self.nb_variants):
            self.nb_high_freq_mutations = np.random.randint(self.min_hfm, self.max_hfm)
            self.high_freq_mutation = np.random.choice(
                range(self.nb_mutation), self.nb_high_freq_mutations, replace=False
            )  # randomly select wich mutations are in high freq for the variant j
            self.variants_matrix[self.high_freq_mutation, j] = 1
        ############# Add error ############
        self.alpha = alpha
        self.variants_matrix_prim = add_error_jit(self.alpha, self.variants_matrix)

        ############# Creat BAM #############
        # initial a matrix that will represent BAM data
        self.BAM = np.zeros((self.size, self.nb_mutation + 1), dtype=np.dtype("i1"))
        # last colum represent the variant of the individual value from 0 to nb_variant,
        # the rest  of colum tell if the individual i has the mutation j value 0 or 1 and -1 if the position has not been read
        
        #Attrebue a variant for each read
        self.BAM[:, -1] = np.random.choice(range(self.nb_variants), p=self.ratio, size=self.size)
        # Attrebute mutations to read i
        for i in range(self.size):
            self.BAM[i, :-1] = np.random.uniform(0, 1, self.nb_mutation) < self.variants_matrix_prim[:, self.BAM[i, -1]]
            
        # Choosing random indexes to put -1, that represente position not been read
        self.degradation_deg = degradation_deg
        self.generate_degradation_profile()
        for col in range(self.nb_mutation):
            index_not_read = np.random.choice(self.size, int(self.size * (self.degradation_profile[col])), replace=False)
            self.BAM[index_not_read,col] = -1

    def generate_observation(self):
        #Calculate nb of reads N where N[i] represente nb of read in position i
        self.N = np.array([self.size - sum(self.BAM[:, col] == -1) for col in range(self.nb_mutation)])
        #Calculate X : nb of mutations observations
        self.X = np.array([sum(self.BAM[:, col] == 1) for col in range(self.nb_mutation)])

        self.maf = self.X/self.N
        self.maf = np.nan_to_num(self.maf, posinf = 0., neginf = 0.)
        
    def solve_with_lstsq(self, alpha=0.):
        #make alpha into account
        self.variants_matrix_sol = add_error_jit(alpha, self.variants_matrix)
        
        try:
            self.sol = np.linalg.lstsq(self.variants_matrix_sol, self.maf)
        except:
            self.maf = np.nan_to_num(self.maf)
            self.sol = np.linalg.lstsq(self.variants_matrix_sol, self.maf)
        
        return self.sol



# EM implementation

In [12]:
#%%
@njit(fastmath=True,parallel=True,cache=True)
def P_x_ik(BAM_k_i, variants_matrix_i_j):
    if BAM_k_i == -1:  # case: X{i,k} was not read
        return 1
    elif BAM_k_i == 1:
        return variants_matrix_i_j
    else:
        return 1 - variants_matrix_i_j

        
                
@njit(fastmath=True,parallel=True,cache=True)
def P_x_k(BAM, variants_matrix, nb_mutation, k, j):
    res = 1
    for i in prange(nb_mutation):
        res *= P_x_ik(BAM[k,i], variants_matrix[i, j])
    return res

@njit(fastmath=True,parallel=True,cache=True)
def dist(v1, v2):
    return np.sum(np.abs(v1 - v2))
    
    
@njit(fastmath=True,parallel=True,cache=True)
def Expectation_v(alpha, size, nb_variants, nb_mutation, BAM, variants_matrix, T, theta_new):
    variants_matrix_prim = variants_matrix - 2*alpha*variants_matrix + alpha
    theta_new_log = [np.log(theta_new_j) for theta_new_j in theta_new]
    result = 0.
    for k in prange(size):
        for j in prange(nb_variants):
            result += T[k, j]*theta_new_log[j]
            for i in prange(nb_mutation):
                result +=  T[k, j]*np.log(P_x_ik(BAM[k, i], variants_matrix_prim[i, j]))
    return -1*result

def call_minimize_scalar(size, nb_variants, nb_mutation, BAM, variants_matrix, T, theta_new):
    result = minimize_scalar(Expectation_v, bounds=(0, 0.5), method='bounded',args=(size, nb_variants, nb_mutation, BAM, variants_matrix, T, theta_new))
    return result.x
# %%
@njit(fastmath=True,parallel=True,cache=True)
def algo_EM(size, nb_variants, nb_mutation, BAM, variants_matrix,alpha=-1, eps=0.0001, max_iter = 100):
    # print("EM_start")
    # initialisation
    
    if alpha == -1 or alpha == 0.: #alpha = -1 means, 2) alpha = 0. cause probleme so change it to a verry small value 
        alpha = 0.00001
        alpha_provided = False
    else:
        alpha_provided = True 
    T = np.zeros((size, nb_variants), np.float64)
    theta = np.array([1.0 / nb_variants for j in range(nb_variants)])
    theta_new = np.array([1.0 / nb_variants for j in range(nb_variants)])  # just to enter in the while loop
    theta_new[0] = theta[0] + 0.02 
    M_prim = variants_matrix - 2*alpha*variants_matrix + alpha
    
    # Start Iteration
    idx_iter = 0
    while ((dist(theta, theta_new) > eps) and (idx_iter < max_iter)):
        ## E step:
        theta = theta_new
        for k in prange(size):
            denominator = 0
            for jj in prange(nb_variants):
                denominator += P_x_k(BAM, M_prim, nb_mutation, k, jj) * theta[jj]
                
            for j in prange(nb_variants):
                T[k, j] = P_x_k(BAM, M_prim, nb_mutation, k, j) * theta[j] / denominator
        
        
        ## M step:
        theta_new = T.sum(axis=0) / size
        # print('new : ',theta_new)
        idx_iter += 1
        
        ## Optimise for alpha:
        if not alpha_provided:
            with objmode(alpha_optimal='float64'):
                alpha_optimal = call_minimize_scalar(size, nb_variants, nb_mutation, BAM, variants_matrix, T, theta_new)
            # print(alpha_optimal)
            M_prim = variants_matrix - 2*alpha_optimal*variants_matrix + alpha_optimal
        else:
            alpha_optimal = alpha
    return (theta_new, alpha_optimal)

In [14]:
p1 = Population(10000, 5, 60, 6, 13)
p1.ratio = [0.35, 0.06, 0.3, 0.01, 0.28]
p1.creat_BAM(alpha=0.2, degradation_deg=5)
p1.generate_observation()
print( '=> ', algo_EM(p1.size, p1.nb_variants, p1.nb_mutation, p1.BAM, p1.variants_matrix))


=>  (array([0.34889, 0.0634 , 0.31496, 0.01037, 0.26239]), 0.1967435045152103)


## Basic test

In [16]:
nb_variants = 8
nb_mutation = 60
size = 100000
a= np.ones((10,8))
## Basic tests
for i in range(5):
    p1 = Population(10000, 5, 60, 6, 13)
    p1.ratio = [0.2, 0.21, 0.3, 0.01, 0.28]
    p1.creat_BAM(alpha = 0.2, degradation_deg = 5)
    p1.generate_observation()
    # print(p1.X_prim)
    np.set_printoptions(precision=5, suppress = True)
    print('EM algo ration is:', algo_EM(p1.size, p1.nb_variants, p1.nb_mutation, p1.BAM, p1.variants_matrix))
    print('naif ration is:', p1.solve_with_lstsq(alpha=0.)[0])
    print('Real ration is: ', p1.ratio)
    print('\n')

EM algo ration is: (array([0.19522, 0.22142, 0.30521, 0.00659, 0.27157]), 0.20338298311915853)
naif ration is: [0.26998 0.22657 0.37195 0.11967 0.2727 ]
Real ration is:  [0.2, 0.21, 0.3, 0.01, 0.28]


EM algo ration is: (array([0.19419, 0.20529, 0.29024, 0.01928, 0.291  ]), 0.20204544240375025)
naif ration is: [0.19184 0.24173 0.34583 0.16401 0.32225]
Real ration is:  [0.2, 0.21, 0.3, 0.01, 0.28]


EM algo ration is: (array([0.18923, 0.21167, 0.29749, 0.02291, 0.2787 ]), 0.1953092816504521)
naif ration is: [0.20627 0.20792 0.2942  0.17983 0.31362]
Real ration is:  [0.2, 0.21, 0.3, 0.01, 0.28]


EM algo ration is: (array([0.22063, 0.19948, 0.29686, 0.00852, 0.2745 ]), 0.19996552135926565)
naif ration is: [0.23915 0.26126 0.28652 0.1295  0.35503]
Real ration is:  [0.2, 0.21, 0.3, 0.01, 0.28]


EM algo ration is: (array([0.20436, 0.21233, 0.30373, 0.01025, 0.26933]), 0.19751370203467167)
naif ration is: [0.24992 0.26858 0.26552 0.12405 0.29555]
Real ration is:  [0.2, 0.21, 0.3, 0.01, 0.28

## Star simulation

# Simulation V2 

In [5]:
def start_one_simulation(size, nb_variants, nb_mutation, min_hfm, ratio, idx):
    global result_N
    global results
    txt = f"{cnt} / {sub_total}" + f"  Simulation : {idx} / {nb_simulation}"
    populaton_actual = Population(size, nb_variants, nb_mutation, min_hfm, min_hfm+1)
    populaton_actual.ratio = ratio
    for degradation_deg in degradation_deg_list:
        for alpha in alpha_list:
            populaton_actual.creat_BAM(alpha, degradation_deg)
            populaton_actual.generate_observation()
            result_EM, alpha_found = algo_EM(populaton_actual.size, populaton_actual.nb_variants, populaton_actual.nb_mutation, populaton_actual.BAM, populaton_actual.variants_matrix, eps=0.0001)
            new_row = [size, nb_variants, nb_mutation, min_hfm, min_hfm+1, alpha, populaton_actual.ratio, degradation_deg, populaton_actual.solve_with_lstsq()[0], populaton_actual.solve_with_lstsq(alpha)[0], algo_EM(populaton_actual.size, populaton_actual.nb_variants, populaton_actual.nb_mutation, populaton_actual.BAM, populaton_actual.variants_matrix, eps=0.0001, alpha=alpha)[0],result_EM, alpha_found]
            results.append(new_row)
            result_N.append([size, nb_variants, nb_mutation, degradation_deg, populaton_actual.N])
            # print(new_row)
    print (txt)

In [6]:
results = []
result_N = []
nb_simulation = 100
size_list = [1000, 10000]
nb_variants_list = [5]
nb_mutation_list = [20, 60]
min_hfm_list = [8]
alpha_list = list(np.arange(0.,0.45, 0.05))
degradation_deg_list = [5]
sub_total=len(size_list)*len(nb_variants_list)*len(nb_mutation_list)*len(min_hfm_list) #Just to help knowing the progress of the semulation while running
cnt = 1
cpt = 0

#Thread pool 
raw_links = set()
good_link = set()
csv_writer_lock = threading.Lock()
MAX_WORKERS = 4 #to be changed in function of your machine proc power
for size in size_list:
    for nb_variants in nb_variants_list:
        ratio = np.random.dirichlet(np.ones(nb_variants) / 1.2, size=1)[0]
        for nb_mutation in nb_mutation_list:
            for min_hfm in min_hfm_list:
                print(f"{cnt} / {sub_total}")
                my_threads = []
                with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
                    tasks = {executor.submit(lambda p: start_one_simulation(*p), [size, nb_variants, nb_mutation, min_hfm, ratio, idx]): idx for idx in range(nb_simulation)}
                cnt += 1


1 / 4
1 / 4  Simulation : 0 / 100
1 / 4  Simulation : 2 / 100
1 / 4  Simulation : 3 / 100
1 / 4  Simulation : 1 / 100
1 / 4  Simulation : 4 / 100
1 / 4  Simulation : 5 / 100
1 / 4  Simulation : 6 / 100
1 / 4  Simulation : 7 / 100
1 / 4  Simulation : 8 / 100
1 / 4  Simulation : 9 / 100
1 / 4  Simulation : 10 / 100
1 / 4  Simulation : 11 / 100
1 / 4  Simulation : 12 / 100
1 / 4  Simulation : 13 / 100
1 / 4  Simulation : 14 / 100
1 / 4  Simulation : 16 / 100
1 / 4  Simulation : 15 / 100
1 / 4  Simulation : 17 / 100
1 / 4  Simulation : 19 / 100
1 / 4  Simulation : 18 / 100
1 / 4  Simulation : 20 / 100
1 / 4  Simulation : 21 / 100
1 / 4  Simulation : 22 / 100
1 / 4  Simulation : 23 / 100
1 / 4  Simulation : 25 / 100
1 / 4  Simulation : 24 / 100
1 / 4  Simulation : 26 / 100
1 / 4  Simulation : 28 / 100
1 / 4  Simulation : 27 / 100
1 / 4  Simulation : 30 / 100
1 / 4  Simulation : 29 / 100
1 / 4  Simulation : 31 / 100
1 / 4  Simulation : 33 / 100
1 / 4  Simulation : 32 / 100
1 / 4  Simulation 

In [10]:
#Export data

results_df = pd.DataFrame(results, columns=['size','nb_variants', 'nb_mutation', 'min_hfm', 'max_hfm', 'alpha', 'ratio', 'degradation_deg','naif_solution', 'naif_solution_alpha_known','EM_solution_alpha_known', 'EM_solution', 'alpha_found'])
results_df.to_csv('result_semulation_EM_v4_temp1.csv', index=False)
results_N_df = pd.DataFrame(result_N, columns=['size','nb_variants', 'nb_mutation', 'degradation_deg', 'populaton_actual.N'])
results_N_df.to_csv('N_result_N_semulation_EM_v4_temp1.csv', index=False)

# Visualization

In [1]:
from ast import literal_eval
import plotly.express as px
import plotly.graph_objects as go

In [5]:
#Import data
data=pd.read_csv('result_semulation_EM_v4_temp1.csv')
# convert columns dtype trom str to their 'real' dtypes
data.infer_objects()
#column that contain lists
data['ratio'] = data.ratio.apply(lambda x: literal_eval(str(x).replace('   ', ',').replace('  ', ',').replace(' ', ',').replace(',,,', ',').replace(',,', ',').replace('[,', '[').replace(',]', ']')))
data['naif_solution'] = data.naif_solution.apply(lambda x: literal_eval(str(x).replace('   ', ',').replace('  ', ',').replace(' ', ',').replace(',,,', ',').replace(',,', ',').replace('[,', '[').replace(',]', ']')))
data['naif_solution_alpha_known'] = data['naif_solution_alpha_known'].apply(lambda x: literal_eval(str(x).replace('   ', ',').replace('  ', ',').replace(' ', ',').replace(',,,', ',').replace(',,', ',').replace('[,', '[').replace(',]', ']')))
data['EM_solution'] = data.EM_solution.apply(lambda x: literal_eval(str(x).replace('   ', ',').replace('  ', ',').replace(' ', ',').replace(',,,', ',').replace(',,', ',').replace('[,', '[').replace(',]', ']')))
data['EM_solution_alpha_known'] = data['EM_solution_alpha_known'].apply(lambda x: literal_eval(str(x).replace('   ', ',').replace('  ', ',').replace(' ', ',').replace(',,,', ',').replace(',,', ',').replace('[,', '[').replace(',]', ']')))
# Sort by alpha 
temp_df = data.sort_values(by='alpha')
temp_df = temp_df[(temp_df['alpha'] == 0.3) & (temp_df['size'] == 10000) & (temp_df['nb_mutation'] == 60)]
temp_df

,size,nb_variants,nb_mutation,min_hfm,max_hfm,alpha,ratio,degradation_deg,naif_solution,naif_solution_alpha_known,EM_solution_alpha_known,EM_solution,alpha_found
3195,10000,5,60,8,9,0.3,"[0.24800529, 0.05149473, 0.43705379, 0.2055608...",5,"[0.24051196, 0.25982457, 0.35936438, 0.3151916...","[0.22976356, 0.0479442, 0.46028101, 0.1930195,...","[0.23099201, 0.05171382, 0.4589883, 0.19317974...","[0.23158413, 0.04991013, 0.46293839, 0.1924323...",0.302968
3534,10000,5,60,8,9,0.3,"[0.24800529, 0.05149473, 0.43705379, 0.2055608...",5,"[0.32771505, 0.2317027, 0.45473492, 0.27294864...","[0.25758525, 0.05801533, 0.42408992, 0.2214980...","[0.2573474, 0.0609611, 0.42475447, 0.21624223,...","[0.25752372, 0.06065145, 0.425241, 0.2166367, ...",0.300682
3467,10000,5,60,8,9,0.3,"[0.24800529, 0.05149473, 0.43705379, 0.2055608...",5,"[0.33328737, 0.28702709, 0.37184354, 0.2346945...","[0.25560192, 0.04883021, 0.43644305, 0.2240250...","[0.24931587, 0.05301952, 0.42870639, 0.2099858...","[0.24930315, 0.05176828, 0.43073164, 0.2100427...",0.301914
3233,10000,5,60,8,9,0.3,"[0.24800529, 0.05149473, 0.43705379, 0.2055608...",5,"[0.35246455, 0.28512395, 0.42220268, 0.3013051...","[0.24135322, 0.05615886, 0.43580831, 0.2089607...","[0.24505377, 0.0646553, 0.43233044, 0.20207088...","[0.24502938, 0.06512408, 0.4320117, 0.20214063...",0.299685
3246,10000,5,60,8,9,0.3,"[0.24800529, 0.05149473, 0.43705379, 0.2055608...",5,"[0.26492824, 0.26225522, 0.44638669, 0.3241975...","[0.22018986, 0.05130789, 0.45772443, 0.2125907...","[0.21831183, 0.05267009, 0.45459121, 0.2045343...","[0.21850763, 0.05175477, 0.45612641, 0.2045553...",0.301684
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3320,10000,5,60,8,9,0.3,"[0.24800529, 0.05149473, 0.43705379, 0.2055608...",5,"[0.30669639, 0.20096909, 0.35711316, 0.2614703...","[0.24597108, 0.05517249, 0.43733179, 0.2226958...","[0.23292093, 0.05580253, 0.43389442, 0.2196556...","[0.23324783, 0.05499084, 0.43525854, 0.2197725...",0.301044
3323,10000,5,60,8,9,0.3,"[0.24800529, 0.05149473, 0.43705379, 0.2055608...",5,"[0.33583055, 0.21248819, 0.26814261, 0.2421262...","[0.2351881, 0.05616511, 0.44102365, 0.19499539...","[0.2326047, 0.07292138, 0.42829233, 0.1992422,...","[0.2323315, 0.07266356, 0.4292419, 0.19937334,...",0.300920
3403,10000,5,60,8,9,0.3,"[0.24800529, 0.05149473, 0.43705379, 0.2055608...",5,"[0.36821742, 0.2760955, 0.416048, 0.24180104, ...","[0.24904185, 0.04529866, 0.43686833, 0.2077919...","[0.24550486, 0.03708986, 0.43808887, 0.2121012...","[0.24558761, 0.03952587, 0.43480564, 0.2099483...",0.295561
3540,10000,5,60,8,9,0.3,"[0.24800529, 0.05149473, 0.43705379, 0.2055608...",5,"[0.33129673, 0.21885593, 0.39067757, 0.3665324...","[0.2435326, 0.06961852, 0.42982971, 0.23150639...","[0.22649112, 0.06789287, 0.43116048, 0.2221809...","[0.22702975, 0.06644338, 0.43472095, 0.2217861...",0.303422


In [6]:
# useful for the visualization
nb_variants = 5
nb_simulation = 100
# alpha_list = list(np.arange(0.,0.5, 0.05))
columns_name = ['naif_solution', 'naif_solution_alpha_known','EM_solution_alpha_known', 'EM_solution']
# Start visualization
for variant_actuel in range(nb_variants):
    temp = []
    for i in range(nb_simulation):
        new_row = [np.array(temp_df[col])[i][variant_actuel] for col in columns_name]
        temp.append(new_row)
    variant = pd.DataFrame(temp, columns=columns_name)
    real_value = list(temp_df.ratio)[0][variant_actuel]
    fig = px.box(variant.melt(), y="value", facet_col="variable", boxmode="overlay", color="variable", labels={'variable': ''}, title=f'Variant {variant_actuel + 1}')
    fig.add_trace(go.Scatter(y=[real_value], mode="markers", name=f"{real_value:.3f} Real value", marker_line_width=2, marker_size=10,marker_symbol=34, marker_color='rgba(0, 0, 0, 1)' ),row='all', col='all', exclude_empty_subplots=True)

    fig.show()